## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-01-11-51-28 +0000,nypost,Mom speaks out on hellish jail cell after bein...,https://nypost.com/2025/09/01/world-news/mom-w...
1,2025-09-01-11-50-09 +0000,bbc,EU chief von der Leyen's plane hit by suspecte...,https://www.bbc.com/news/articles/c9d07z1439zo...
2,2025-09-01-11-46-57 +0000,nypost,Mom charged with murdering baby she ‘intention...,https://nypost.com/2025/09/01/us-news/mom-char...
3,2025-09-01-11-45-47 +0000,bbc,Peak rail fares scrapped on ScotRail trains,https://www.bbc.com/news/articles/c98lzygd5vjo...
4,2025-09-01-11-45-15 +0000,nyt,Hundreds Dead After 6.0-Magnitude Earthquake H...,https://www.nytimes.com/2025/08/31/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
122,trump,30
63,china,11
136,killed,11
371,he,10
73,new,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
159,2025-08-31-19-58-46 +0000,nypost,Illinois Gov. Pritzker floats conspiracy theor...,https://nypost.com/2025/08/31/us-news/pritzker...,66
176,2025-08-31-18-12-00 +0000,wsj,The Supreme Court has flirted with a flurry of...,https://www.wsj.com/politics/policy/trump-tari...,65
183,2025-08-31-17-11-30 +0000,nypost,"Trump says he, Putin and Zelensky will meet to...",https://nypost.com/2025/08/31/world-news/trump...,65
145,2025-08-31-21-45-30 +0000,nypost,Trump mulls ‘GREAT’ plan to pay Palestinians $...,https://nypost.com/2025/08/31/us-news/trump-mu...,62
70,2025-09-01-08-32-33 +0000,bbc,Putin says he reached 'understandings' with Tr...,https://www.bbc.com/news/articles/c0qljy44553o...,60


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
159,66,2025-08-31-19-58-46 +0000,nypost,Illinois Gov. Pritzker floats conspiracy theor...,https://nypost.com/2025/08/31/us-news/pritzker...
151,49,2025-08-31-21-23-44 +0000,nypost,Israel begins heavy airstrikes ahead of Gaza C...,https://nypost.com/2025/08/31/world-news/israe...
126,33,2025-08-31-23-36-14 +0000,nypost,11-year-old boy allegedly shot and killed by T...,https://nypost.com/2025/08/31/us-news/11-year-...
0,32,2025-09-01-11-51-28 +0000,nypost,Mom speaks out on hellish jail cell after bein...,https://nypost.com/2025/09/01/world-news/mom-w...
76,30,2025-09-01-07-50-54 +0000,bbc,BYD shares slide as China's EV price war hits ...,https://www.bbc.com/news/articles/cx2pr1wwq44o...
12,29,2025-09-01-11-12-47 +0000,nypost,Driver arrested after crashing car into gates ...,https://nypost.com/2025/09/01/world-news/drive...
15,28,2025-09-01-11-00-00 +0000,wsj,"The cash benefits of the new tax law, signed b...",https://www.wsj.com/politics/policy/companies-...
21,26,2025-09-01-10-35-44 +0000,wapo,Man found dead in pool of blood at Burning Man...,https://www.washingtonpost.com/nation/2025/09/...
67,25,2025-09-01-08-53-15 +0000,nypost,Rosie O’Donnell apologizes for calling trans M...,https://nypost.com/2025/09/01/us-news/rosie-od...
176,25,2025-08-31-18-12-00 +0000,wsj,The Supreme Court has flirted with a flurry of...,https://www.wsj.com/politics/policy/trump-tari...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
